In [31]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_csv("clean_total.csv")

In [33]:
df.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,booking_date,release_date,bail_type,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry,age
0,679537,301831.0,1974-04-02,MALE,WHITE,M,SHAWNEE,KS,66226.0,15CR00773,...,2015-04-01,2015-04-01,CA,10000.0,1,"MAJOR DEPRESSIVE DISORDER, SINGLE EPISODE, MOD...",OTHERS,1,0,41.0
1,431835,296410.0,1971-01-01,MALE,WHITE,S,OLATHE,KS,66062.0,13CR00129,...,2013-01-19,2013-01-22,SUR,10000.0,1,SOCIAL PHOBIA,OTHERS,1,1,42.0
2,431835,296410.0,1971-01-01,MALE,WHITE,S,OLATHE,KS,66062.0,14CR00932,...,2014-05-02,2014-05-02,SUR,5000.0,1,SOCIAL PHOBIA,OTHERS,1,1,43.0
3,658145,350849.0,1997-04-12,FEMALE,WHITE,S,EDGERTON,KS,66021.0,15CR02335,...,2015-10-07,2015-10-07,SUR,10000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,18.0
4,655865,370683.0,1984-08-03,MALE,WHITE,S,OLATHE,KS,66062.0,13CR00468,...,2013-03-03,2013-03-03,SUR,1000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,29.0


In [34]:
df.shape

(29504, 24)

In [35]:
test = df[df['re_entry']==1]

In [36]:
good = df[df['re_entry']==0]

In [40]:
test.shape

(12895, 24)

In [39]:
good.shape

(16609, 24)

In [37]:
test['booking_date'] = pd.to_datetime(test['booking_date'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
test['release_date'] = pd.to_datetime(test['release_date'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [41]:
def within_frame(df, id_num='dedupe_id', timestamp='booking_date', col_name='re-entry', duration=365):
    df = df.sort_values(by=[id_num, timestamp])
    df['{}'.format(col_name)] = df.groupby(id_num)[timestamp].diff()
    df['{}'.format(col_name)] = df['{}'.format(col_name)].apply(lambda x: x.days)
    df['within_{}'.format(duration)] = np.where(df['{}'.format(col_name)]>duration, 1, 0)    

    return df

In [42]:
test = test.sort_values(by=['dedupe_id', 'booking_date'])

In [43]:
test.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,booking_date,release_date,bail_type,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry,age
20701,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,11CR01499,...,2011-07-05,2011-07-05,PR,2500.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,21.0
20702,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,11CR02750,...,2011-11-20,2012-10-22,SUR,2500.0,0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,22.0
20703,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR01019,...,2014-05-02,2014-05-19,SUR,1500.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0
20704,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR02622,...,2014-12-03,2014-12-04,SUR,5000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0
10703,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,14DV00270,...,2014-03-04,2014-03-05,PR,1500.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,36.0


In [44]:
test['1yr'] = test.groupby('dedupe_id')['booking_date'].diff()

In [45]:
test['stayed'] = test['release_date'] - test['booking_date']

In [46]:
test['stayed'].head()

20701     0 days
20702   337 days
20703    17 days
20704     1 days
10703     1 days
Name: stayed, dtype: timedelta64[ns]

In [47]:
test['1yr'].head()

20701        NaT
20702   138 days
20703   894 days
20704   215 days
10703        NaT
Name: 1yr, dtype: timedelta64[ns]

In [48]:
test[['mni_no', '1yr', 'stayed']].head()

,mni_no,1yr,stayed
20701,494625,NaT,0 days
20702,494625,138 days,337 days
20703,494625,894 days,17 days
20704,494625,215 days,1 days
10703,673675,NaT,1 days


In [49]:
test.stayed = test.stayed.shift(1)

In [50]:
test[['mni_no', 'booking_date', 'release_date', '1yr', 'stayed']].head()

,mni_no,booking_date,release_date,1yr,stayed
20701,494625,2011-07-05,2011-07-05,NaT,NaT
20702,494625,2011-11-20,2012-10-22,138 days,0 days
20703,494625,2014-05-02,2014-05-19,894 days,337 days
20704,494625,2014-12-03,2014-12-04,215 days,17 days
10703,673675,2014-03-04,2014-03-05,NaT,1 days


In [51]:
test = test[test['1yr'].notnull()]

In [52]:
test[['mni_no', 'booking_date', 'release_date', '1yr', 'stayed']].head()

,mni_no,booking_date,release_date,1yr,stayed
20702,494625,2011-11-20,2012-10-22,138 days,0 days
20703,494625,2014-05-02,2014-05-19,894 days,337 days
20704,494625,2014-12-03,2014-12-04,215 days,17 days
10704,673675,2015-03-26,2015-03-27,387 days,1 days
10705,673675,2016-02-24,2016-04-28,335 days,1 days


In [53]:
test['after_released'] = test['1yr'] -test['stayed'] 

In [54]:
test.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry,age,1yr,stayed,after_released
20702,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,11CR02750,...,2500.0,0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,22.0,138 days,0 days,138 days
20703,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR01019,...,1500.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0,894 days,337 days,557 days
20704,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR02622,...,5000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0,215 days,17 days,198 days
10704,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,15DV00382,...,1000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,37.0,387 days,1 days,386 days
10705,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,16DV00234,...,2500.0,0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,38.0,335 days,1 days,334 days


In [55]:
#Convert the days type to integer 
test['after_released'] = test['after_released'].astype('timedelta64[D]')

In [56]:
test.shape

(8246, 27)

In [57]:
test["within_one"] = np.where(test["after_released"] <= 365, 1, 0)

In [58]:
test["within_two"] = np.where(test["after_released"] <= 365*2, 1, 0)

In [59]:
test.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,pri_dx_value,refferal_source,mh_treatment,re_entry,age,1yr,stayed,after_released,within_one,within_two
20702,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,11CR02750,...,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,22.0,138 days,0 days,138.0,1,1
20703,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR01019,...,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0,894 days,337 days,557.0,0,1
20704,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR02622,...,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0,215 days,17 days,198.0,1,1
10704,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,15DV00382,...,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,37.0,387 days,1 days,386.0,0,1
10705,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,16DV00234,...,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,38.0,335 days,1 days,334.0,1,1


In [60]:
good['after_released']=0

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [62]:
good['within_one'] = 0

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [63]:
good['within_two']=0


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [64]:
good.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry,age,after_released,within_one,within_two
0,679537,301831.0,1974-04-02,MALE,WHITE,M,SHAWNEE,KS,66226.0,15CR00773,...,10000.0,1,"MAJOR DEPRESSIVE DISORDER, SINGLE EPISODE, MOD...",OTHERS,1,0,41.0,0,0,0
3,658145,350849.0,1997-04-12,FEMALE,WHITE,S,EDGERTON,KS,66021.0,15CR02335,...,10000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,18.0,0,0,0
4,655865,370683.0,1984-08-03,MALE,WHITE,S,OLATHE,KS,66062.0,13CR00468,...,1000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,29.0,0,0,0
5,631990,467807.0,1975-02-18,MALE,WHITE,M,LEAWOOD,KS,66209.0,11DV01549,...,3500.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,36.0,0,0,0
6,510773,103594.0,1991-06-11,MALE,WHITE,S,OVERLAND PARK,KS,66212.0,15DV00580,...,2500.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,24.0,0,0,0


In [65]:
test= test.drop(['stayed', '1yr'], axis=1)

In [66]:
test.head()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry,age,after_released,within_one,within_two
20702,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,11CR02750,...,2500.0,0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,22.0,138.0,1,1
20703,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR01019,...,1500.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0,557.0,0,1
20704,494625,131.0,1990-05-31,MALE,WHITE,S,OLATHE,KS,66062.0,14CR02622,...,5000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,24.0,198.0,1,1
10704,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,15DV00382,...,1000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,37.0,386.0,0,1
10705,673675,309.0,1978-04-07,MALE,WHITE,S,OLATHE,KS,66061.0,16DV00234,...,2500.0,0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,1,38.0,334.0,1,1


In [67]:
df = test.append([good])

In [68]:
df.shape

(24855, 27)

In [70]:
df.tail()

,mni_no,dedupe_id,dob,sex,race,mar_stat,city,state,zip,case_no,...,bail_amt,bailed_out,pri_dx_value,refferal_source,mh_treatment,re_entry,age,after_released,within_one,within_two
29499,624233,544732.0,1981-05-10,MALE,WHITE,S,LENEXA,KS,66215.0,15CR02999,...,5000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,34.0,0.0,0,0
29500,393726,674256.0,1984-03-07,MALE,WHITE,S,OVERLAND PARK,KS,66212.0,10CR01234,...,10000.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,26.0,0.0,0,0
29501,697108,35713.0,1988-05-11,FEMALE,WHITE,M,GIDEON,MO,63848.0,15DV00978,...,2500.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,27.0,0.0,0,0
29502,354499,359078.0,1986-06-19,MALE,WHITE,S,KANSAS CITY,MO,64137.0,14DV01165,...,0.0,1,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,28.0,0.0,0,0
29503,606679,550016.0,1981-02-04,MALE,WHITE,D,CLARKRANGE,TN,38553.0,10CR01863,...,2500.0,0,UNSPECIFIED AFFECTIVE PSYCHOSIS,NaN,0,0,30.0,0.0,0,0


In [71]:
df.to_csv('final_data.csv', index=False)